In [1]:
%matplotlib inline

In [2]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Helper functions to make the code more readable.



In [5]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)


#这里增加未登录字处理
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else to_ix["<UNK>"] for w in seq] #to_ix.has_key(x)
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

Create model



In [6]:

class BiLSTM_CRF(nn.Module):
 
    # 初始化参数
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        # 词嵌入维度
        self.embedding_dim = embedding_dim
        # BiLSTM 隐藏层维度
        self.hidden_dim = hidden_dim
        # 词典的大小
        self.vocab_size = vocab_size
        # tag到数字的映射
        self.tag_to_ix = tag_to_ix
        # tag个数
        self.tagset_size = len(tag_to_ix)
        # num_embeddings (int)：vocab_size 词典的大小
        # embedding_dim (int)：embedding_dim 嵌入向量的维度，即用多少维来表示一个符号
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        # input_size: embedding_dim 输入数据的特征维数，通常就是embedding_dim(词向量的维度)
        # hidden_size: hidden_dim LSTM中隐藏层的维度
        # num_layers：循环神经网络的层数
        # 默认使用偏置，默认不用dropout
        # bidirectional = True 用双向LSTM
        # 设定为单层双向
        # 隐藏层设定为指定维度的一半，便于后期拼接
        # // 表示整数除法,返回不大于结果的一个最大的整数
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        # 将BiLSTM提取的特征向量映射到特征空间，即经过全连接得到发射分数
        # in_features: hidden_dim 每个输入样本的大小
        # out_features:tagset_size 每个输出样本的大小
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        # 转移矩阵的参数初始化，transition[i,j]代表的是从第j个tag转移到第i个tag的转移分数
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        # 初始化所有其他tag转移到START_TAG的分数非常小，即不可能由其他tag转移到START_TAG
        # 初始化STOP_TAG转移到所有其他的分数非常小，即不可能有STOP_TAG转移到其他tag
        # CRF的转移矩阵,T[i,j]表示从j标签转移到i标签，
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        # 初始化LSTM的参数
        self.hidden = self.init_hidden()
 
    # 使用随机正态分布初始化LSTM的h0和c0
    # 否则模型自动初始化为零值，维度为[num_layers*num_directions, batch_size, hidden_dim]
    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))
 
    # 计算归一化因子Z(x)
    def _forward_alg(self, feats):
        '''
        输入：发射矩阵(emission score)，实际上就是LSTM的输出
        sentence的每个word经BiLSTM后对应于每个label的得分
        输出：所有可能路径得分之和/归一化因子/配分函数/Z(x)
        '''
        # 通过前向算法递推计算
        # 初始化1行 tagset_size列的嵌套列表
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # 初始化step 0 即START位置的发射分数，START_TAG取0其他位置取-10000
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        # 包装到一个变量里面以便自动反向传播
        forward_var = init_alphas
        # 迭代整个句子
        # feats:形似[[....], 每个字映射到tag的发射概率，
        #        [....],
        #        [....]]
        for feat in feats:
            # 存储当前时间步下各tag得分
            alphas_t = []
            for next_tag in range(self.tagset_size):
                # 取出当前tag的发射分数(与之前时间步的tag无关)，扩展成tag维
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # 取出当前tag由之前tag转移过来的转移分数
                trans_score = self.transitions[next_tag].view(1, -1)
                # 当前路径的分数：之前时间步分数+转移分数+发射分数
                next_tag_var = forward_var + trans_score + emit_score
                # 对当前分数取log-sum-exp
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            # 更新forward_var 递推计算下一个时间步
            # torch.cat 默认按行添加
            forward_var = torch.cat(alphas_t).view(1, -1)
        # 考虑最终转移到STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        # 对当前分数取log-sum-exp
        scores = log_sum_exp(terminal_var)
        return scores
 
    # 通过BiLSTM提取特征
    def _get_lstm_features(self, sentence):
        # 初始化LSTM的h0和c0
        self.hidden = self.init_hidden()
        # 使用之前构造的词嵌入为语句中每个词（word_id）生成向量表示
        # 并将shape改为[seq_len, 1(batch_size), embedding_dim]
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        # LSTM网络根据输入的词向量和初始状态h0和c0
        # 计算得到输出结果lstm_out和最后状态hn和cn
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        # 转换为词 - 标签([seq_len, tagset_size])表
        # 可以看作为每个词被标注为对应标签的得分情况，即维特比算法中的发射矩阵
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
 
    # 计算一个tag序列路径的得分
    def _score_sentence(self, feats, tags):
        # feats发射分数矩阵
        # 计算给定tag序列的分数，即一条路径的分数
        score = torch.zeros(1)
        # tags前面补上一个句首标签便于计算转移得分
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        # 循环用于计算给定tag序列的分数
        for i, feat in enumerate(feats):
            # 递推计算路径分数：转移分数+发射分数
            # T[i,j]表示j转移到i
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        # 加上转移到句尾的得分，便得到了gold_score
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
 
    # veterbi解码，得到最优tag序列
    def _viterbi_decode(self, feats):
        '''
        :param feats: 发射分数矩阵
        :return:
        '''
        # 便于之后回溯最优路径
        backpointers = []
        # 初始化viterbi的forward_var变量
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        # forward_var表示每个标签的前向状态得分，即上一个词被打作每个标签的对应得分值
        forward_var = init_vvars
        # 遍历每个时间步时的发射分数
        for feat in feats:
            # 记录当前词对应每个标签的最优转移结点
            # 保存当前时间步的回溯指针
            bptrs_t = []
            # 与bptrs_t对应，记录对应的最优值
            # 保存当前时间步的viterbi变量
            viterbivars_t = []
            # 遍历每个标签，求得当前词被打作每个标签的得分
            # 并将其与当前词的发射矩阵feat相加，得到当前状态，即下一个词的前向状态
            for next_tag in range(self.tagset_size):
                # transitions[next_tag]表示每个标签转移到next_tag的转移得分
                # forward_var表示每个标签的前向状态得分，即上一个词被打作每个标签的对应得分值
                # 二者相加即得到当前词被打作next_tag的所有可能得分
                # 维特比算法记录最优路径时只考虑上一步的分数以及上一步的tag转移到当前tag的转移分数
                # 并不取决于当前的tag发射分数
                next_tag_var = forward_var + self.transitions[next_tag]
                # 得到上一个可能的tag到当前tag中得分最大值的tag位置索引id
                best_tag_id = argmax(next_tag_var)
                # 将最优tag的位置索引存入bptrs_t
                bptrs_t.append(best_tag_id)
                # 添加最优tag位置索引对应的值
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # 更新forward_var = 当前词的发射分数feat + 前一个最优tag当前tag的状态下的得分
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            # 回溯指针记录当前时间步各个tag来源前一步的最优tag
            backpointers.append(bptrs_t)
        # forward_var表示每个标签的前向状态得分
        # 加上转移到句尾标签STOP_TAG的转移得分
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        # 得到标签STOP_TAG前一个时间步的最优tag位置索引
        best_tag_id = argmax(terminal_var)
        # 得到标签STOP_TAG当前最优tag对应的分数值
        path_score = terminal_var[0][best_tag_id]
        # 根据过程中存储的转移路径结点，反推最优转移路径
        # 通过回溯指针解码出最优路径
        best_path = [best_tag_id]
        # best_tag_id作为线头，反向遍历backpointers找到最优路径
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # 去除START_TAG
        start = best_path.pop()
        # 最初的转移结点一定是人为构建的START_TAG，删除，并根据这一点确认路径正确性
        assert start == self.tag_to_ix[START_TAG]
        # 最后将路径倒序即得到从头开始的最优转移路径best_path
        best_path.reverse()
        return path_score, best_path
 
    # 损失函数loss
    def neg_log_likelihood(self, sentence, tags):
        # 得到句子对应的发射分数矩阵
        feats = self._get_lstm_features(sentence)
        # 通过前向算法得到归一化因子Z(x)
        forward_score = self._forward_alg(feats)
        # 得到tag序列的路径得分
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
 
    # 输入语句序列得到最佳tag路径及其得分
    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # 从BiLSTM获得发射分数矩阵
        lstm_feats = self._get_lstm_features(sentence)
        # 使用维特比算法进行解码，计算最佳tag路径及其得分
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


## 自己的数据

In [68]:
f=open("/content/完整数据.data","r",encoding="utf-8")
sentences=[]
labels=[]


In [69]:
seq_data=[]
seq_label=[]
for i in f.readlines():
    
    i=i.replace("\n", "")
    lst=i.split(" ")
    
    if len(lst)==4:
        seq_data.append(lst[0])
        seq_label.append(lst[3])
        
    else: #语料中是空行分隔句子
        sent=" ".join(seq_data)
        seq_data.clear()
        sentences.append(sent)
        
        label=" ".join(seq_label)
        seq_label.clear()
        labels.append(label)   

    print(i.split(" "))

流式输出内容被截断，只能显示最后 5000 行内容。
['予', 'v', 'O', 'O']
['惩处', 'v', 'O', 'O']
['鉴于', 'p', 'O', 'O']
['被告人', 'n', 'O', 'O']
['冉某某', 'v', 'O', 'O']
['肇事', 'v', 'O', 'O']
['后', 'nd', 'O', 'O']
['未', 'd', 'O', 'O']
['离开', 'v', 'O', 'O']
['现场', 'nl', 'O', 'O']
['归案', 'v', 'O', 'O']
['后', 'nd', 'O', 'O']
['如实', 'd', 'O', 'B-P']
['供述', 'v', 'O', 'I-P']
['自己', 'r', 'O', 'I-P']
['犯罪事实', 'v', 'O', 'E-P']
['庭审', 'v', 'O', 'O']
['中', 'nd', 'O', 'O']
['认罪', 'v', 'O', 'B-P']
['态度', 'n', 'O', 'I-P']
['好', 'a', 'O', 'E-P']
['亲属', 'n', 'O', 'O']
['积极', 'a', 'O', 'B-P']
['赔偿', 'v', 'O', 'I-P']
['被害人', 'n', 'O', 'I-P']
['经济损失', 'v', 'O', 'E-P']
['行为', 'n', 'O', 'O']
['取得', 'v', 'O', 'B-P']
['被害人', 'n', 'O', 'I-P']
['近', 'a', 'O', 'I-P']
['亲属', 'n', 'O', 'I-P']
['及', 'c', 'O', 'I-P']
['伤者', 'n', 'O', 'I-P']
['本人', 'r', 'O', 'I-P']
['谅解', 'v', 'O', 'E-P']
['宣告', 'v', 'O', 'O']
['缓刑', 'v', 'O', 'O']
['对', 'p', 'O', 'O']
['所', 'u', 'O', 'O']
['居住', 'v', 'O', 'O']
['社区', 'n', 'O', 'O']
['没有', 'v', 'O', 'O']
['重大', 'a

In [70]:
len(sentences),len(labels)

(694, 694)

In [71]:
sentences[0:5]

['1 被告人 吴永飞 在 公路 上 行驶 发生 交通事故 致 被害人 死亡 经 交警大队 交通事故 认定书 认定 被告人 负 事故 主要责任 行为 构成 交通肇事罪 公诉机关 指控 被告人 吴永飞 犯 交通肇事罪 名 成立 本院 予以支持 应 以 交通肇事罪 追究 刑事责任 被告人 在 事故 发生 后 驾车逃逸 应 依法 在 三年 以上 七年 以下 有期徒刑 幅度 内 判处 刑罚 鉴于 被告人 当庭 自愿认罪 可 酌定 从轻处罚 赔偿 被害人 经济损失 取得 被害人 亲属 谅解 亦可 从轻处罚 公诉机关 量刑 建议 符合 法律 规定 本院 予以采纳 鉴于 被告人 具有 悔罪表现 可 依法 适用 缓刑 据 上 之 规定 判决如下 被告人 吴永飞 犯 交通肇事罪 判处 有期徒刑 三年 缓刑 四年',
 '2 被告人 张某某 造成 重大 交通事故 致 一 人 死亡 二 人 受伤 被告人 张某某 行为 构成 交通肇事罪 公诉机关 指控 犯罪事实 及 罪名成立 本院 予以 确认 被告人 张某某 如实 供述 犯罪 行为 积极 对 被害人 张某甲 家属 进行 民事赔偿 故可 酌情 从轻处罚 被告人 张某某 辩护人 辩护观点 正确 本院 予以采纳 为 维护 交通 管理 秩序 保护 人民群众 生命 财产 安全 不 受 侵害 经合 议庭 评议 七十二 条 之 规定 判决如下 被告人 张某某 犯 交通肇事罪 判处 有期徒刑 二年 缓刑 三年',
 '3 被告人 李骥博 致 一 人 死亡 事故 发生 负 事故 主要责任 行为 构成 交通肇事罪 公诉机关 指控 犯罪 成立 予以支持 李骥博案 发 后 明知 他人 报警 并 在 现场 等候 无 抗拒 抓捕 行为 到案 后 如实 交待 犯罪事实 系自首 可 从轻处罚 李骥博 在 案发 后 主动 赔偿 被害人 近 亲属 全部 经济损失 并 取得 被害人 近 亲属 谅解 可 酌予 从轻处罚 根据 社会 调查 李骥博 此次 犯罪系 初犯 对 判处 缓刑 在 居住 宜阳县 三乡乡 流渠村 不 会 产生 重大 不良 影响 所 居住 三乡乡 流渠村 村民 委员会 愿意 协助 司法 机关 对 进行 社区 矫正故 可 对 适用 缓刑 根据 本案 事实 情节 以及 社会 危害 程度 之 规定 判决如下 被告人 李骥博 犯 交通肇事罪 判处 有期徒刑 十个月 宣告

In [72]:
labels[0:5]

['O O O O O O O O S-T O B-N E-N O O O O O O O O S-D O O S-K O O O O O S-K O O O O O O S-K O O O O O O O S-N O O O O O O O O O O O O O O B-P E-P O O O B-P I-P E-P B-P I-P I-P E-P O O O O O O O O O O O O B-P E-P O O O O O O O O S-R O O O S-K S-R B-R E-R B-R E-R',
 'O O O O B-T E-T O B-P I-P E-P B-P I-P E-P O O O O S-K O O O O O O O O O O B-P I-P O O S-P O O O O O S-P O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O S-R O O O S-K S-R B-R E-R B-R E-R',
 'O O O O B-N I-N E-N O O O O O O O S-K O O O O O O O O O O O O B-P I-P E-P B-P I-P I-P E-P O O B-P I-P E-P S-P O O O O O O B-P E-P O O O B-P E-P O S-P O O O S-P O O O O O O O O O S-P O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O S-R O O O S-K S-R B-R E-R O B-R E-R',
 'O O O O S-T O B-N E-N O B-K E-K O O O O B-T E-T O O O O O O O O O B-T I-T I-T I-T E-T O O O O O O O O O O O O O O O O O O O S-P B-P E-P O O O O O O O O O B-P I-P E-P O O O O O S-R O O O O S-K S-R B-R I-R I-R I-R I-R I-R E-R O O 

In [73]:
data=[]

for i in range(len(sentences)):
    data.append( (sentences[i].split(), labels[i].split()) )

In [74]:
len(data)

694

In [14]:
# training_data[0:5]
# training_data[0][0]

In [80]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {}
word_to_ix = {"<UNK>":0} #生字给id=0

for sentence, tags in data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag]=len(tag_to_ix)
        

In [81]:
len(word_to_ix),len(tag_to_ix)

(4416, 29)

In [83]:
tag_to_ix

{'B-D': 21,
 'B-K': 17,
 'B-N': 2,
 'B-P': 7,
 'B-R': 11,
 'B-T': 13,
 'E-D': 22,
 'E-K': 18,
 'E-N': 3,
 'E-P': 8,
 'E-R': 12,
 'E-T': 14,
 'EP-P': 25,
 'I-D': 28,
 'I-K': 23,
 'I-N': 16,
 'I-P': 9,
 'I-R': 20,
 'I-T': 19,
 'N-P': 27,
 'O': 0,
 'O-K': 24,
 'S-D': 4,
 'S-K': 5,
 'S-N': 6,
 'S-P': 15,
 'S-R': 10,
 'S-T': 1,
 'W-P': 26}

In [84]:
tag_to_ix[START_TAG]=29
tag_to_ix[STOP_TAG]=30

In [85]:
tag_to_ix

{'<START>': 29,
 '<STOP>': 30,
 'B-D': 21,
 'B-K': 17,
 'B-N': 2,
 'B-P': 7,
 'B-R': 11,
 'B-T': 13,
 'E-D': 22,
 'E-K': 18,
 'E-N': 3,
 'E-P': 8,
 'E-R': 12,
 'E-T': 14,
 'EP-P': 25,
 'I-D': 28,
 'I-K': 23,
 'I-N': 16,
 'I-P': 9,
 'I-R': 20,
 'I-T': 19,
 'N-P': 27,
 'O': 0,
 'O-K': 24,
 'S-D': 4,
 'S-K': 5,
 'S-N': 6,
 'S-P': 15,
 'S-R': 10,
 'S-T': 1,
 'W-P': 26}

In [ ]:
# word_to_ix

In [86]:
from sklearn.model_selection import train_test_split

training_data,test_data=train_test_split(data,test_size=0.2, random_state=0)

In [87]:
len(training_data),len(test_data)

(555, 139)

### 训练

In [109]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 64
EPOCHS=1

In [110]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
# optimizer=optim.Adam(model.parameters(),lr=0.01,weight_decay=1e-4)

In [111]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
import time


for epoch in range(EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data
    print("epoch %d =============" % epoch)
    time_start = time.time()
    
    count=1
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix) 
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
        
        if count%10==0:
            print("iter %d: loss %f" %(count,loss))
        count+=1
        
    time_end=time.time()
    print("time used: %d s" % (time_end-time_start)  )

epoch 0 =============
iter 10: loss 99.035950
iter 20: loss 98.264801
iter 30: loss 145.798523
iter 40: loss 158.954895
iter 50: loss 110.535309
iter 60: loss 150.253601
iter 70: loss 109.609802
iter 80: loss 95.015076
iter 90: loss 58.761536
iter 100: loss 105.885254
iter 110: loss 54.558289
iter 120: loss 35.930115
iter 130: loss 36.612793
iter 140: loss 60.935913
iter 150: loss 39.789429
iter 160: loss 15.436401
iter 170: loss 62.787231
iter 180: loss 26.058533
iter 190: loss 22.231201
iter 200: loss 30.037048
iter 210: loss 77.336060
iter 220: loss 23.889648
iter 230: loss 35.288940
iter 240: loss 35.273438
iter 250: loss 98.256958
iter 260: loss 17.048523
iter 270: loss 11.996582
iter 280: loss 10.351562
iter 290: loss 18.217957
iter 300: loss 26.842072
iter 310: loss 11.590027
iter 320: loss 10.394897
iter 330: loss 26.605835
iter 340: loss 15.053223
iter 350: loss 8.111633
iter 360: loss 36.677063
iter 370: loss 19.915405
iter 380: loss 47.382202
iter 390: loss 37.871277
iter 40

### 测试

In [121]:
def compare(y, y_pred):
    error_index = []
    if len(y) == len(y_pred):
        for i in range(0, len(y)):
            if y[i] != y_pred[i]:
                error_index.append(i)

    return error_index
    # print("error_index:",error_index)
    

In [127]:

sumloss=0

# Check predictions before training
with torch.no_grad():  
    
    # for pair in test_data[0:10]: #抽取部分看看效果
    for pair in test_data: #抽取部分看看效果
        sentence=pair[0]
        tag=pair[1]
        
        precheck_sent = prepare_sequence(sentence, word_to_ix)
        precheck_tags = torch.tensor([tag_to_ix[t] for t in tag], dtype=torch.long) 
        score,y_pred=model(precheck_sent)
            
        print(sentence)
        print(tag)
        print("实际tagid:",list(precheck_tags.numpy()))
        print("预测tagid:",y_pred)
        errorindex=compare(precheck_tags,y_pred)
        sumloss= sumloss+(len(errorindex)/len(precheck_tags))
        print(sumloss)
        print("error_index:",errorindex)
        print("====================")

    print(sumloss/len(test_data))


['494', '被告人', '周某', '驾驶', '安全机', '件', '不', '符合', '制动', '要求', '车辆', '在', '道路', '上', '行驶', '发生', '重大', '交通事故', '致', '一', '人', '死亡', '负', '事故', '全部', '责任', '行为', '构成', '交通肇事罪', '公诉机关', '指控', '罪名', '正确', '本院', '予以支持', '在', '事故', '发生', '主动', '报警', '并', '如实', '供述', '自己', '犯罪事实', '是', '自首', '依法', '可以', '从轻处罚', '案', '发', '后', '被告人', '周某', '积极', '赔偿', '损失', '并', '取得', '被害人', '亲属', '谅解', '可', '酌情', '从轻处罚', '根据', '被告人', '周某', '犯罪事实', '犯罪', '性质', '情节', '及', '本案', '客观', '情况', '对', '适用', '缓刑', '不致', '再', '危害', '社会', '可以', '宣告', '缓刑', '据此', '之', '规定', '判决如下', '被告人', '周某', '犯', '交通肇事罪', '判处', '有期徒刑', '一年', '缓刑', '二年']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-T', 'E-T', 'O', 'B-N', 'I-N', 'E-N', 'O', 'O', 'B-D', 'E-D', 'O', 'O', 'S-K', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-P', 'E-P', 'O', 'B-P', 'E-P', 'O', 'O', 'B-P', 'E-P', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-P', 'E-P', 'O', 'O', 'S-P', 'O', 'O', 'S-P', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

### 预测

In [128]:
with torch.no_grad():  
    precheck_sent = prepare_sequence("浙江淘宝网络有限公司法定代表人发生变更，张勇卸任，由85后的蒋凡接任。", word_to_ix)
    score,y_pred=model(precheck_sent)
    print("预测tagid:",y_pred)

预测tagid: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [138]:
with torch.no_grad():  
    precheck_sent = prepare_sequence("造成致一人死亡交通事故肇事后逃逸负事故主要责任行为构成交通肇事罪", word_to_ix)
    score,y_pred=model(precheck_sent)
    print("预测tagid:",y_pred)

预测tagid: [0, 0, 0, 2, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
